In [1]:
DATA_NAME = 'segmentAnything-full' 
TRANSFORM = 'wavelet-vertical'
CHANNEL = 'green'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_green_vertical_wavelet_full_segmentAnything.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9,10
2,8230.60,-0.06,-0.00,0.05,0.00,-0.01,0.00,-0.0,-0.00
3,-0.06,2326.36,-0.02,0.01,-0.01,0.00,0.01,0.0,0.00
4,-0.00,-0.02,657.66,0.01,0.00,0.00,-0.00,-0.0,0.00
5,0.05,0.01,0.01,171.96,0.00,-0.00,0.00,0.0,0.00
6,0.00,-0.01,0.00,0.00,42.03,-0.00,-0.00,0.0,0.00
7,-0.01,0.00,0.00,-0.00,-0.00,9.80,0.00,0.0,0.00
8,0.00,0.01,-0.00,0.00,-0.00,0.00,2.21,0.0,0.00
9,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.5,-0.00
10,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.0,0.12


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9,10
2,1.00000,-0.00001,-0.00000,0.00004,0.00000,-0.00003,0.00001,-0.00002,-0.00000
3,-0.00001,1.00000,-0.00001,0.00001,-0.00003,0.00002,0.00007,0.00001,0.00001
4,-0.00000,-0.00001,1.00000,0.00004,0.00000,0.00000,-0.00005,-0.00001,0.00002
5,0.00004,0.00001,0.00004,1.00000,0.00000,-0.00002,0.00007,0.00001,0.00002
6,0.00000,-0.00003,0.00000,0.00000,1.00000,-0.00001,-0.00000,0.00003,0.00001
7,-0.00003,0.00002,0.00000,-0.00002,-0.00001,1.00000,0.00002,0.00000,0.00001
8,0.00001,0.00007,-0.00005,0.00007,-0.00000,0.00002,1.00000,0.00004,0.00001
9,-0.00002,0.00001,-0.00001,0.00001,0.00003,0.00000,0.00004,1.00000,-0.00001
10,-0.00000,0.00001,0.00002,0.00002,0.00001,0.00001,0.00001,-0.00001,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.11555282656331663

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[7.53797234e+06 5.92266415e+05 4.63544515e+04 3.08184278e+03
 1.76866268e+02 9.02657046e+00 4.08981226e-01 1.65707614e-02
 1.91250060e-35]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7,8
0,0.999214,-0.038285,-0.009921,-0.002571,-0.000629,-0.000148,-0.000033,-0.000008,-0.000002
1,0.037780,0.998250,-0.044174,-0.010556,-0.002570,-0.000597,-0.000132,-0.000031,-0.000007
2,0.011434,0.043196,0.997829,-0.047126,-0.010658,-0.002473,-0.000560,-0.000128,-0.000029
3,0.003459,0.012307,0.045945,0.997421,-0.052354,-0.011383,-0.002544,-0.000583,-0.000135
4,0.001013,0.003590,0.012722,0.050935,0.996599,-0.061981,-0.013050,-0.002971,-0.000691
5,0.000295,0.001039,0.003678,0.014107,0.060006,0.994790,-0.079221,-0.016960,-0.003927
6,0.000087,0.000305,0.001090,0.004153,0.017054,0.075563,0.989450,-0.119609,-0.025739
7,0.000029,0.000100,0.000356,0.001359,0.005560,0.023887,0.108921,0.966604,-0.230674
8,0.000014,0.000049,0.000173,0.000662,0.002712,0.011635,0.051684,0.225996,0.972682


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0007858948134034094,
 0.0017501033821303302,
 0.0021711004203005535,
 0.002579212715152601,
 0.0034014823974576336,
 0.005210335665319588,
 0.01054970006128575,
 0.03339646147438802,
 0.027317527520969165]